# Capstone 4

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.formula.api as smf
import sklearn.metrics
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

Topic: Speaking as a DA from a business that wants to perform better compared to others in the same country, and uncover what they're doing better that we can potentially adopt. I'd also like to make a prediction on products sold based on seller gender, country, # of followers, etc!

# Loading datasets

In [7]:
# Countries in each dataset written in French. When inspecting on Excel, noticed encryption/accents. Saving to UTF-8,
# and using IFERROR + VLOOKUP to translate country names

In [2]:
# Dealing with encrypted and accented characters:
# Open Excel
# Click “File” and “New”
# Click on the “Data” tab
# Click “From Text” and select the CSV file
# Select “Delimited”
# For “File origin”, select “65001 : Unicode (UTF-8)”
# Click “Next”
# Select “Comma”
# Click “Finish”
# Excel should now show you the CSV file and display the characters correctly.

In [8]:
# Loading revised datasets

In [9]:
# Dataset of users and information about their social shopping/behavior

users = pd.read_csv('~/Documents/JONATHANAYALA-CAPSTONE4/DATA/users.csv', encoding='utf8')

users.head(5)

,identifierHash,type,country,eng_country,language,socialNbFollowers,socialNbFollows,socialProductsLiked,productsListed,productsSold,...,civilityTitle,hasAnyApp,hasAndroidApp,hasIosApp,hasProfilePicture,daysSinceLastLogin,seniority,seniorityAsMonths,seniorityAsYears,countryCode
0,-1.097900e+18,user,Royaume-Uni,United Kingdom,en,147,10,77,26,174,...,mr,True,False,True,True,11,3196,106.53,8.88,gb
1,2.347570e+18,user,Monaco,Monaco,en,167,8,2,19,170,...,mrs,True,False,True,True,12,3204,106.80,8.90,mc
2,6.870940e+18,user,France,France,fr,137,13,60,33,163,...,mrs,True,False,True,False,11,3203,106.77,8.90,fr
3,-4.640270e+18,user,Etats-Unis,United States,en,131,10,14,122,152,...,mrs,True,False,True,False,12,3198,106.60,8.88,us
4,-5.175830e+18,user,Etats-Unis,United States,en,167,8,0,25,125,...,mrs,False,False,False,True,22,2854,95.13,7.93,us


In [10]:
# Comparison of seller accounts by country and gender

sellers = pd.read_csv('~/Documents/JONATHANAYALA-CAPSTONE4/DATA/sellers.csv', encoding='utf8')

sellers.head(5)

,country,eng_country,sex,nbsellers,meanproductssold,meanproductslisted,meansellerpassrate,totalproductssold,totalproductslisted,meanproductsbought,meanproductswished,meanproductsliked,totalbought,totalwished,totalproductsliked,meanfollowers,meanfollows,percentofappusers,percentofiosusers,meanseniority
0,Allemagne,Germany,Female,116,4.03,2.72,27.33,468,315,3.05,34.66,35.28,354,4021,4092,9.5,8.9,54.0,49.0,3060.336207
1,Allemagne,Germany,Male,34,2.00,1.00,19.15,68,34,1.62,3.38,31.79,55,115,1081,7.8,8.4,79.0,64.0,3089.058824
2,Arménie,Armenia,Female,1,0.00,25.00,0.00,0,25,0.00,0.00,1.00,0,0,1,4.0,8.0,NaN,NaN,3201.000000
3,Australie,Australia,Female,18,0.94,1.33,10.44,17,24,6.11,17.72,209.28,110,319,3767,7.5,9.3,55.0,55.0,3103.666667
4,Australie,Australia,Male,3,6.00,4.00,33.33,18,12,8.00,24.00,38.33,24,72,115,12.7,8.3,66.0,66.0,3085.666667


In [11]:
# Stats about countries with top sellers. Top sellers those with >= 20 products sold
# They represent ~ the top 10% of all seller accounts in the dataset
# (Seller accounts are those who successfully sold at least 1 product)
# These are just insights extracted from the main file!

topsellers = pd.read_csv('~/Documents/JONATHANAYALA-CAPSTONE4/DATA/topsellers.csv', encoding='utf8')

topsellers.head(5)

,country,eng_country,sellers,topsellers,topsellerratio,femalesellersratio,topfemalesellersratio,femalesellers,malesellers,topfemalesellers,...,topmeanproductssold,topmeanproductslisted,meanproductssold,meanproductslisted,meanofflinedays,topmeanofflinedays,meanfollowers,meanfollowing,topmeanfollowers,topmeanfollowing
0,Taiwan,Taiwan,1,1,100.0,100.0,100.0,1,0,1,...,57.00,56.00,57.000000,56.000000,11.0,11.0,83.0,8.0,83.0,8.0
1,Slovaquie,Slovakia,2,1,50.0,0.0,0.0,0,2,0,...,27.00,14.00,14.000000,7.000000,17.0,15.0,10.5,8.5,15.0,8.0
2,Lettonie,Latvia,4,2,50.0,100.0,100.0,4,0,2,...,40.50,18.00,20.750000,9.000000,120.3,11.5,21.0,52.3,38.0,98.5
3,Bulgarie,Bulgaria,9,4,44.4,66.7,100.0,6,3,4,...,36.25,17.25,18.888889,9.111111,98.3,19.0,28.6,31.6,46.3,19.0
4,Chypre,Cyprus,4,1,25.0,100.0,100.0,4,0,1,...,41.00,66.00,14.000000,20.250000,17.3,11.0,21.3,10.3,39.0,17.0


# EDA

In [12]:
# users

In [13]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98913 entries, 0 to 98912
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   identifierHash       98913 non-null  float64
 1   type                 98913 non-null  object 
 2   country              98913 non-null  object 
 3   eng_country          98913 non-null  object 
 4   language             98913 non-null  object 
 5   socialNbFollowers    98913 non-null  int64  
 6   socialNbFollows      98913 non-null  int64  
 7   socialProductsLiked  98913 non-null  int64  
 8   productsListed       98913 non-null  int64  
 9   productsSold         98913 non-null  int64  
 10  productsPassRate     98913 non-null  float64
 11  productsWished       98913 non-null  int64  
 12  productsBought       98913 non-null  int64  
 13  gender               98913 non-null  object 
 14  civilityGenderId     98913 non-null  int64  
 15  civilityTitle        98913 non-null 

In [14]:
users.describe()

,identifierHash,socialNbFollowers,socialNbFollows,socialProductsLiked,productsListed,productsSold,productsPassRate,productsWished,productsBought,civilityGenderId,daysSinceLastLogin,seniority,seniorityAsMonths,seniorityAsYears
count,9.891300e+04,98913.000000,98913.000000,98913.000000,98913.000000,98913.000000,98913.000000,98913.000000,98913.000000,98913.000000,98913.000000,98913.000000,98913.000000,98913.000000
mean,-6.692037e+15,3.432269,8.425677,4.420743,0.093304,0.121592,0.812303,1.562595,0.171929,1.773993,581.291236,3063.771870,102.125583,8.510424
std,5.330807e+18,3.882383,52.839572,181.030569,2.050144,2.126895,8.500205,25.192793,2.332266,0.428679,208.855888,168.298621,5.609735,0.467863
min,-9.223100e+18,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,11.000000,2852.000000,95.070000,7.920000
25%,-4.622890e+18,3.000000,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,572.000000,2857.000000,95.230000,7.940000
50%,-1.337990e+15,3.000000,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,694.000000,3196.000000,106.530000,8.880000
75%,4.616390e+18,3.000000,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,702.000000,3201.000000,106.700000,8.890000
max,9.223330e+18,744.000000,13764.000000,51671.000000,244.000000,174.000000,100.000000,2635.000000,405.000000,3.000000,709.000000,3205.000000,106.830000,8.900000


In [15]:
# sellers

In [16]:
sellers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73 entries, 0 to 72
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   country              70 non-null     object 
 1   eng_country          70 non-null     object 
 2   sex                  72 non-null     object 
 3   nbsellers            73 non-null     int64  
 4   meanproductssold     73 non-null     float64
 5   meanproductslisted   73 non-null     float64
 6   meansellerpassrate   73 non-null     float64
 7   totalproductssold    73 non-null     int64  
 8   totalproductslisted  73 non-null     int64  
 9   meanproductsbought   73 non-null     float64
 10  meanproductswished   73 non-null     float64
 11  meanproductsliked    73 non-null     float64
 12  totalbought          73 non-null     int64  
 13  totalwished          73 non-null     int64  
 14  totalproductsliked   73 non-null     int64  
 15  meanfollowers        73 non-null     float

In [17]:
sellers.describe()

,nbsellers,meanproductssold,meanproductslisted,meansellerpassrate,totalproductssold,totalproductslisted,meanproductsbought,meanproductswished,meanproductsliked,totalbought,totalwished,totalproductsliked,meanfollowers,meanfollows,percentofappusers,percentofiosusers,meanseniority
count,73.000000,73.000000,73.000000,73.000000,73.000000,73.000000,73.000000,73.000000,73.000000,73.000000,73.000000,73.000000,73.000000,73.000000,66.000000,65.000000,73.000000
mean,111.739726,4.984110,4.177671,26.382329,494.260274,379.273973,3.266575,40.527397,72.186986,225.616438,1874.424658,6601.931507,12.583562,16.475342,68.015152,61.492308,3070.617187
std,412.253126,8.091924,7.484078,21.453931,1809.983142,1410.134078,5.211880,113.551020,134.051376,824.771679,7067.534787,26250.135826,12.190946,25.093557,19.716836,22.041242,100.929973
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,8.000000,20.000000,12.000000,2855.000000
25%,2.000000,1.200000,1.000000,0.000000,3.000000,3.000000,0.000000,1.130000,6.330000,0.000000,4.000000,15.000000,7.500000,8.200000,57.500000,50.000000,3043.909091
50%,5.000000,2.790000,2.250000,29.460000,24.000000,14.000000,1.290000,7.000000,27.750000,15.000000,80.000000,333.000000,9.500000,9.000000,66.000000,57.000000,3084.600000
75%,22.000000,5.250000,4.150000,37.000000,107.000000,82.000000,3.330000,26.400000,69.990000,77.000000,817.000000,1081.000000,13.400000,12.300000,75.000000,66.000000,3130.600000
max,2719.000000,57.000000,56.000000,98.000000,12027.000000,9229.000000,24.670000,707.000000,792.000000,5490.000000,45611.000000,160647.000000,83.000000,157.000000,100.000000,100.000000,3204.000000


In [18]:
# topsellers

In [19]:
topsellers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   country                 19 non-null     object 
 1   eng_country             19 non-null     object 
 2   sellers                 19 non-null     int64  
 3   topsellers              19 non-null     int64  
 4   topsellerratio          19 non-null     float64
 5   femalesellersratio      19 non-null     float64
 6   topfemalesellersratio   19 non-null     float64
 7   femalesellers           19 non-null     int64  
 8   malesellers             19 non-null     int64  
 9   topfemalesellers        19 non-null     int64  
 10  topmalesellers          19 non-null     int64  
 11  countrysoldratio        19 non-null     float64
 12  bestsoldratio           18 non-null     float64
 13  toptotalproductssold    19 non-null     int64  
 14  totalproductssold       19 non-null     int6

In [20]:
topsellers.describe()

,sellers,topsellers,topsellerratio,femalesellersratio,topfemalesellersratio,femalesellers,malesellers,topfemalesellers,topmalesellers,countrysoldratio,...,topmeanproductssold,topmeanproductslisted,meanproductssold,meanproductslisted,meanofflinedays,topmeanofflinedays,meanfollowers,meanfollowing,topmeanfollowers,topmeanfollowing
count,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,...,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000
mean,99.578947,6.631579,20.968421,78.778947,80.963158,75.368421,24.210526,5.105263,1.526316,2.032632,...,50.026688,30.124714,12.966926,8.582865,122.631579,23.757895,19.752632,17.389474,55.310526,56.389474
std,178.948570,9.804820,24.430174,22.167182,26.052025,131.976097,47.390317,7.194865,2.874571,1.616621,...,34.374679,19.263995,13.502800,12.208144,73.153940,20.889453,17.128448,14.551552,36.883012,103.303425
min,1.000000,1.000000,3.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.690000,...,20.000000,0.000000,3.722222,1.142857,11.000000,11.000000,7.000000,8.000000,15.000000,3.000000
25%,6.000000,1.000000,6.450000,74.300000,71.800000,5.500000,0.500000,1.000000,0.000000,1.230000,...,28.500000,18.500000,5.167766,2.930594,62.600000,11.000000,10.800000,8.700000,36.650000,8.550000
50%,13.000000,2.000000,10.100000,78.700000,85.700000,10.000000,3.000000,1.000000,1.000000,1.590000,...,41.000000,21.380952,8.126801,5.500000,121.600000,12.700000,15.100000,10.200000,46.300000,14.500000
75%,112.000000,5.500000,22.500000,92.850000,100.000000,88.000000,24.000000,4.500000,1.000000,2.015000,...,56.700000,39.500000,14.000000,7.109375,172.800000,28.500000,20.400000,19.750000,58.300000,36.500000
max,713.000000,35.000000,100.000000,100.000000,100.000000,523.000000,190.000000,23.000000,12.000000,7.310000,...,170.000000,71.333333,57.000000,56.000000,266.100000,79.400000,83.000000,54.600000,167.000000,429.300000
